In [1]:
import ee
import geemap

In [2]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

# Functions

In [3]:
# Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask) \
        .divide(10000) \
        .copyProperties(image, ['system:time_start'])  # this guy is important!

# Mask out water
def maskWater(image):
    return image.updateMask(waterMask.select('water_mask').lt(1))

# Function to filter images by NDSI_Snow_Cover value < 5
def maskS2snow(image):

    mask = image.select('MSK_SNWPRB').lt(0.009) \
    
    return image.updateMask(mask) \
            .copyProperties(image, ['system:time_start'])  # this guy is important!

# Make an NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi).copyProperties(image, ['system:time_start'])

# Function to get yearly statistics for the chosen index
def annual_images(y):
    range_year = ee.Filter.calendarRange(y, y, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(lambda image: image.addBands(image.metadata('system:time_start').divide(3.154e10)))) # Needed for linear regression 
    
    # Print out the number of images in the ImageCollection for each year
    num_images = filtered_dataset.size()
    
    
    # Choose the reducer based on the analysis choice
    if analysis == 'mean':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        )
    elif analysis == 'min' or analysis == 'max':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        )
    elif analysis == 'median':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.median(),
            sharedInputs=True
        )

    # Use the combined reducer to get the statistics
    stats = filtered_dataset.reduce(reducer)
    return stats.set('year', y).set('num', num_images)

def intrayear(index_collection):
    range_year = ee.Filter.calendarRange(start_year, end_year, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(lambda image: image.addBands(image.metadata('system:time_start').divide(3.154e10)))) # Needed for linear regression 
    
    num_images = filtered_dataset.size()

    # return filtered_dataset
    return filtered_dataset.set('year', start_year).set('num', num_images)

# Build collection

In [4]:
# Get water mask
waterMask = (
    ee.ImageCollection('MODIS/006/MOD44W') 
    .filter(ee.Filter.date('2015-01-01', '2015-01-02')) 
    .select('water_mask') \
    .first()
)
# Get Sentinel 2 harmonized images
dataset = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                #   // Sentinel 2 harmonized data only available for certain time
                  .filter(ee.Filter.calendarRange(2019,2023,'year'))
                #   // Filter by month. Be mindful of snow! 
                  .filter(ee.Filter.calendarRange(6,9,'month'))
                #   // Pre-filter to get less cloudy granules.
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                #   // .filterBounds(geometry)
                #   // This one's Toolik
                  .filterBounds(ee.Geometry.Point(-149.5427, 68.6267).buffer(500))
                #   // This one's Russian tree tracks
                #   // .filterBounds(ee.Geometry.Point(133.16008, 66.82386).buffer(1000))
                  .map(maskS2clouds)
                  .map(maskS2snow)
                  .map(maskWater)
                  .map(addNDVI)
)

# Do analysis

In [7]:
# Pick your index
index = 'NDVI'
# Pick your reducer
analysis = 'max'  # Choose 'mean', 'median', 'min', or 'max' for analysis

# In case you want to limit beyond your ImageCollection
start_year = 2021
end_year = 2021
start_month = 6
end_month = 8

# Pick your index
index_collection = dataset.select(index)  # Choose your index collection

# Generate list of years
years = ee.List.sequence(start_year, end_year)

if start_year == end_year:
    intrayear_collection = intrayear(index_collection)

    item = intrayear_collection.getInfo()
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

    # Get linear fit to pixelwise trend of annual max NDVI
    trend = intrayear_collection.select(['system:time_start',
                                index
                                ]).reduce(ee.Reducer.linearFit())
    
else:

    # Map over years to get yearly statistics
    yearwise_ndvi = years.map(annual_images)

    for item in yearwise_ndvi.getInfo():
        print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

        yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

        # Get linear fit to pixelwise trend of annual max NDVI
        trend = yearCompCol.select(['system:time_start_mean',
                                    f'{index}_{analysis}'
                                    ]).reduce(ee.Reducer.linearFit())



Year: 2021 Number of images: 5


In [12]:
Map = geemap.Map()

# Map.addLayer(dataset, {
#     'min':0.0,
#     'max':0.3,
#     'bands': ['B4', 'B3', 'B2']}, 'RGB')

image_limit = 5
for image_id in intrayear(dataset).aggregate_array("system:index").getInfo()[0:image_limit]:
        image = intrayear(dataset).filterMetadata("system:index", "equals", image_id).first()
        
        image_RGB = image.select('B4', 'B3', 'B2') 
        # image_RGB = image.select('B4') 
        RGB_vis_params = {'min': 0.0, 'max': 0.3}
        Map.addLayer(image_RGB, RGB_vis_params, ee.Image(image).date().format('yyyy-MM-dd').getInfo(), True)

for image_id in intrayear_collection.aggregate_array("system:index").getInfo()[0:image_limit]:
        image = intrayear_collection.filterMetadata("system:index", "equals", image_id).first()
        date_string = ee.Image(image).date().format('yyyy-MM-dd').getInfo()

        Map.addLayer(image.select('NDVI'), {}, f'{date_string}_NDVI', False)

Map.addLayer(trend.select('scale'),
              {
                      'min':-1.0, 'max':3.0,
            'palette': ['red', 'white', 'blue']
            },
 'trend')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…